In [32]:
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re
import string

In [9]:
spot = sp.Spotify(client_credentials_manager=sp.oauth2.SpotifyClientCredentials(client_id='', client_secret=''))

In [10]:
artist = 'frank ocean'
results = spot.search(q=f'artist:{artist}', type='artist')

In [11]:
uri = results['artists']['items'][0]['uri']

In [18]:
song_list = []

for i in range(10):
    song_list.append(spot.artist_top_tracks(uri)['tracks'][i]['name'])

song_list

['Pink + White',
 'Lost',
 'Novacane',
 'Ivy',
 'Slide (feat. Frank Ocean & Migos)',
 'Thinkin Bout You',
 'Nights',
 'White Ferrari',
 'In My Room',
 'Pyramids']

In [21]:
query = f'azlyrics {artist} {song_list[0]}'

search_list = []

for j in search(query, num_results=5):
    search_list.append(j)

search_list

['https://www.azlyrics.com/lyrics/frankocean/pinkwhite.html',
 'https://www.lyrics.com/lyric/33235750/Frank+Ocean/Pink+++White',
 'https://azlyrics.biz/f/frank-ocean-lyrics/frank-ocean-pink-white-lyrics/',
 'https://genius.com/Frank-ocean-pink-white-lyrics',
 'http://www.songlyrics.com/frank-ocean/pink-white-lyrics/']

In [22]:
url = search_list[0]

In [24]:
result = requests.get(url)
content = result.content
content

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="viewport" content="width=device-width, initial-scale=1">\r\n<meta name="description" content="Frank Ocean &quot;Pink + White&quot;: Yeah, yeah, uh (woo!) Yeah, yeah, yeah That\'s the way every day goes Every time we\'ve no control If..."> \r\n<meta name="keywords" content="Pink + White lyrics, Frank Ocean Pink + White lyrics, Frank Ocean lyrics">\r\n<meta name="robots" content="noarchive">\r\n<meta property="og:image" content="//www.azlyrics.com/az_logo_tr.png">\r\n<title>Frank Ocean - Pink + White Lyrics | AZLyrics.com</title>\r\n\r\n<link rel="canonical" href="https://www.azlyrics.com/lyrics/frankocean/pinkwhite.html" />\r\n<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css">\r\n<link rel="stylesheet" href="/local/az.css">\r\n\r\n<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements

In [27]:
soup = BeautifulSoup(content, 'html.parser')
result.close()

In [28]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Frank Ocean &quot;Pink + White&quot;: Yeah, yeah, uh (woo!) Yeah, yeah, yeah That's the way every day goes Every time we've no control If..." name="description"/>
  <meta content="Pink + White lyrics, Frank Ocean Pink + White lyrics, Frank Ocean lyrics" name="keywords"/>
  <meta content="noarchive" name="robots"/>
  <meta content="//www.azlyrics.com/az_logo_tr.png" property="og:image"/>
  <title>
   Frank Ocean - Pink + White Lyrics | AZLyrics.com
  </title>
  <link href="https://www.azlyrics.com/lyrics/frankocean/pinkwhite.html" rel="canonical"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.4/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="/local/az.css" rel="stylesheet"/>
  <!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media quer

In [29]:
tag = soup.find('div', {'class': 'ringtone'}).find_next_sibling('div')

In [30]:
lyrics = tag.text
print(lyrics[0:138])



Yeah, yeah, uh (woo!)
Yeah, yeah, yeah

That's the way every day goes
Every time we've no control
If the sky is pink and white
If the g


In [33]:
lyrics = re.sub('\\[.*?\\]', '', lyrics)
lyrics = re.sub('\n\n', '\n', lyrics)
print(lyrics[0:138])



Yeah, yeah, uh (woo!)
Yeah, yeah, yeah
That's the way every day goes
Every time we've no control
If the sky is pink and white
If the gr


In [34]:
no_punc = [char for char in lyrics if char not in string.punctuation]
no_punc[0:10]

['\n', '\r', '\n', 'Y', 'e', 'a', 'h', ' ', 'y', 'e']

In [35]:
lyrics = ''.join(no_punc)
print(lyrics[0:120])



Yeah yeah uh woo
Yeah yeah yeah
Thats the way every day goes
Every time weve no control
If the sky is pink and white



In [36]:
words = []
for word in lyrics.split(' '):
    if not '\n' in word:
        words.append(word)
    else:
        phrase = word.split('\n')
        for i in range(len(phrase)-1):
            if phrase[i] != '':
                words.append(phrase[i])
                words.append('\n')
        if phrase[-1] != '':
            words.append(phrase[-1])         

words

['\r',
 '\n',
 'Yeah',
 'yeah',
 'uh',
 'woo',
 '\n',
 'Yeah',
 'yeah',
 'yeah',
 '\n',
 'Thats',
 'the',
 'way',
 'every',
 'day',
 'goes',
 '\n',
 'Every',
 'time',
 'weve',
 'no',
 'control',
 '\n',
 'If',
 'the',
 'sky',
 'is',
 'pink',
 'and',
 'white',
 '\n',
 'If',
 'the',
 'ground',
 'is',
 'black',
 'and',
 'yellow',
 '\n',
 'Its',
 'the',
 'same',
 'way',
 'you',
 'showed',
 'me',
 '\n',
 'Nod',
 'my',
 'head',
 'dont',
 'close',
 'my',
 'eyes',
 '\n',
 'Halfway',
 'on',
 'a',
 'slow',
 'move',
 '\n',
 'Its',
 'the',
 'same',
 'way',
 'you',
 'showed',
 'me',
 '\n',
 'If',
 'you',
 'could',
 'fly',
 'then',
 'youd',
 'feel',
 'south',
 '\n',
 'Up',
 'norths',
 'getting',
 'cold',
 'soon',
 '\n',
 'The',
 'way',
 'it',
 'is',
 'were',
 'on',
 'land',
 '\n',
 'Still',
 'Im',
 'someone',
 'to',
 'hold',
 'true',
 '\n',
 'Keep',
 'you',
 'cool',
 'when',
 'its',
 'still',
 'alive',
 '\n',
 'Wont',
 'let',
 'you',
 'down',
 'when',
 'its',
 'all',
 'ruin',
 '\n',
 'Just',
 'the',
